In [9]:
import pandas as pd
import tiktoken
from dotenv import load_dotenv
# from utils.embeddings_utils import get_embeddings
from openai import OpenAI
from typing import List

load_dotenv()

embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
client = OpenAI(max_retries=5)
max_tokens = 8000

# Helper functions

In [10]:
def get_embedding(text: str, model="text-embedding-3-small", **kwargs) -> List[float]:
    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")

    response = client.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding

In [11]:
path = "data/reviews.csv"
df = pd.read_csv(path, index_col=[0])
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df.dropna(inplace=True)
df['combined'] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

,Time,ProductId,UserId,Score,Summary,Text,combined
Id,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...


In [12]:
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# Pass reviews that are too long to embedding model
df['n_tokens'] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

1000

In [13]:
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, model=embedding_model))
df.to_csv("data/fine_food_reviews_with_embeddings_1k.csv")